# Analytische Vaardigheden




Importeren van de dependencies


In [159]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import datetime
pd.options.mode.chained_assignment = None

## Het inladen van de dataset
Eerst laden we de dataset in

In [160]:
indexes = pd.read_csv('GSPC.csv')
indexes.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1950-01-03,16.66,16.66,16.66,16.66,16.66,1260000
1,1950-01-04,16.85,16.85,16.85,16.85,16.85,1890000
2,1950-01-05,16.93,16.93,16.93,16.93,16.93,2550000
3,1950-01-06,16.98,16.98,16.98,16.98,16.98,2010000
4,1950-01-09,17.08,17.08,17.08,17.08,17.08,2520000


## Het schoonmaken
Dan gaan we de dataset schoonmaken en alle kolommen droppen die we niet meer gaan gebruiken. Ook maken we van de kolomnamen kleine letters want dat is handig!

In [161]:
indexes['date'] = pd.to_datetime(indexes['Date'])
indexes['open'] = indexes['Open']
columns_to_keep = [
    'date',
    'open'
]
indexes = indexes[columns_to_keep]
indexes.head()

,date,open
0,1950-01-03,16.66
1,1950-01-04,16.85
2,1950-01-05,16.93
3,1950-01-06,16.98
4,1950-01-09,17.08


Nu gaan we de startjaar en eindjaar bepalen

In [162]:
startjaar = 2013
duur = 4
eindjaar = startjaar + duur
eindjaar

2017

We bepalen de rente

In [163]:
rente = 1.00

Nu gaan we een nieuwe Dataframe maken waar al onze resultaten in komen, we maken er eerst eentje met alle datums waar we gaan investeren. 

In [176]:
jaren = pd.date_range(str(startjaar) + "-09-01", periods=duur + 1, freq='AS-SEP')
df = pd.DataFrame({'date': jaren, 'schuld': 0.0})
df.head()

,date,schuld
0,2013-09-01,0.0
1,2014-09-01,0.0
2,2015-09-01,0.0
3,2016-09-01,0.0
4,2017-09-01,0.0


Nu gaan we de maandelijkse studiefinanciering bepalen, deze is te vinden op de site van DUO.

In [165]:
lening = 486.08
beurs = 396.39
collegegeldkrediet = 173.59
totaal = lening + beurs + collegegeldkrediet
totaal

1056.06

Dat is wat we per maand kunnen krijgen van Ome duo. Per jaar is dat dus:

In [166]:
totaal = totaal * 12
totaal

12672.72

Op 1 september van het eerste jaar hebben we nog geen geld, want dan beginnen we pas, dus dan is de schuld 0

In [167]:
df['schuld'][0] = 0
df.head()

,date,schuld
0,2013-09-01,0.0
1,2014-09-01,0.0
2,2015-09-01,0.0
3,2016-09-01,0.0
4,2017-09-01,0.0


Doordat er in de dataset niet altijd een 1 september te vinden is, gaan we kijken wat de eerstvolgende datum is dat er wel een waarde beschikbaar is

In [227]:
def get_day_value(date):
    global df, indexes
    if len(indexes[indexes['date'] != date]) != len(indexes) - 1:
        return get_day_value(date + datetime.timedelta(days=1))
    else:
        return indexes[indexes['date'] == date]

for index, row in df.iterrows():
    df['date'][index] = get_day_value(row['date']).iloc[0]['date']
df.head()
df = pd.merge(df, indexes, on='date')

In [228]:
df.head()

,date,schuld,open
0,2013-09-03,0.0,1635.949951
1,2014-09-02,0.0,2004.069946
2,2015-09-01,0.0,1970.089966
3,2016-09-01,0.0,2171.330078
4,2017-09-01,0.0,2474.419922
